## Imports

In [79]:
import geopandas as gpd
import json
import fiona
from bokeh.io import curdoc
from bokeh.models import GeoJSONDataSource, LogColorMapper
from bokeh.palettes import Magma256 as palette
from bokeh.plotting import figure,ColumnDataSource,output_file,show,output_notebook
from shapely.geometry import Polygon, Point, MultiPoint, MultiPolygon
from shapely.prepared import prep
from bokeh.tile_providers import get_provider, Vendors
from bokeh.layouts import column, row, widgetbox, layout
from bokeh.models import Slider, Toggle,DateSlider,DateRangeSlider,sliders,callbacks
from bokeh.models.callbacks import CustomJS
from bokeh.models import SaveTool
import pandas as pd
from geopy.geocoders import Nominatim
import numpy as np
import matplotlib.pyplot as plt
from datetime import date

In [80]:
fires_df = pd.read_csv('../Data/fire_incidents.csv')
fires_df.drop(columns='Unnamed: 0',inplace = True)
fires_df.head()

,alarm_box_borough,alarm_box_location,alarm_box_number,alarm_level_index_description,alarm_source_description_tx,citycouncildistrict,communitydistrict,communityschooldistrict,congressionaldistrict,dispatch_response_seconds_qy,...,incident_datetime,incident_response_seconds_qy,incident_travel_tm_seconds_qy,ladders_assigned_quantity,other_units_assigned_quantity,policeprecinct,starfire_incident_id,valid_dispatch_rspns_time_indc,valid_incident_rspns_time_indc,zipcode
0,QUEENS,95 AVE & 110 ST,6028,Initial Alarm,PD Link/Medical,28.0,409.0,27.0,5.0,5,...,2018-12-31T23:59:59.000,0,0,0,0,102.0,1836560280150880,N,N,11419.0
1,BROOKLYN,19 AVE & 84 ST,2875,Initial Alarm,EMS Link/Medical,47.0,311.0,20.0,11.0,4,...,2018-12-31T23:58:14.000,266,262,0,0,62.0,1836528750141240,N,Y,11214.0
2,BRONX,RANDALL AV & CROSS BX EXPY EB SVC RD,4139,Initial Alarm,EMS Link/Medical,13.0,210.0,8.0,14.0,4,...,2018-12-31T23:57:05.000,0,0,0,0,45.0,1836541390120920,N,N,10465.0
3,BRONX,E TREMONT AVE & WEBSTER AVE,2982,Initial Alarm,Private Fire Alarm,15.0,206.0,9.0,15.0,136,...,2018-12-31T23:57:02.000,353,217,1,0,48.0,1836529820120920,N,Y,10457.0
4,MANHATTAN,AUDUBON AVE & W 177 ST,1726,Initial Alarm,Private Fire Alarm,10.0,112.0,6.0,13.0,39,...,2018-12-31T23:56:49.000,242,203,2,1,33.0,1836517260211290,N,Y,10033.0


In [81]:
fires_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 29 columns):
alarm_box_borough                 object
alarm_box_location                object
alarm_box_number                  int64
alarm_level_index_description     object
alarm_source_description_tx       object
citycouncildistrict               float64
communitydistrict                 float64
communityschooldistrict           float64
congressionaldistrict             float64
dispatch_response_seconds_qy      int64
engines_assigned_quantity         int64
first_activation_datetime         object
first_assignment_datetime         object
first_on_scene_datetime           object
highest_alarm_level               object
incident_borough                  object
incident_classification           object
incident_classification_group     object
incident_close_datetime           object
incident_datetime                 object
incident_response_seconds_qy      int64
incident_travel_tm_seconds

In [82]:
fires_df['incident_datetime'] = pd.to_datetime(fires_df['incident_datetime'])

In [83]:
fires_df['incident_month'] = [i.month for i in fires_df['incident_datetime']]

In [84]:
fires_df['incident_year'] = [i.year for i in fires_df['incident_datetime']]

In [85]:
fires_df['zipcode'] = [str(i)[0:5] for i in fires_df['zipcode']]

In [86]:
fires_df.head()

,alarm_box_borough,alarm_box_location,alarm_box_number,alarm_level_index_description,alarm_source_description_tx,citycouncildistrict,communitydistrict,communityschooldistrict,congressionaldistrict,dispatch_response_seconds_qy,...,incident_travel_tm_seconds_qy,ladders_assigned_quantity,other_units_assigned_quantity,policeprecinct,starfire_incident_id,valid_dispatch_rspns_time_indc,valid_incident_rspns_time_indc,zipcode,incident_month,incident_year
0,QUEENS,95 AVE & 110 ST,6028,Initial Alarm,PD Link/Medical,28.0,409.0,27.0,5.0,5,...,0,0,0,102.0,1836560280150880,N,N,11419,12,2018
1,BROOKLYN,19 AVE & 84 ST,2875,Initial Alarm,EMS Link/Medical,47.0,311.0,20.0,11.0,4,...,262,0,0,62.0,1836528750141240,N,Y,11214,12,2018
2,BRONX,RANDALL AV & CROSS BX EXPY EB SVC RD,4139,Initial Alarm,EMS Link/Medical,13.0,210.0,8.0,14.0,4,...,0,0,0,45.0,1836541390120920,N,N,10465,12,2018
3,BRONX,E TREMONT AVE & WEBSTER AVE,2982,Initial Alarm,Private Fire Alarm,15.0,206.0,9.0,15.0,136,...,217,1,0,48.0,1836529820120920,N,Y,10457,12,2018
4,MANHATTAN,AUDUBON AVE & W 177 ST,1726,Initial Alarm,Private Fire Alarm,10.0,112.0,6.0,13.0,39,...,203,2,1,33.0,1836517260211290,N,Y,10033,12,2018


In [87]:
fires_by_zipcode_month_year = fires_df.groupby(by = ['incident_year','incident_month','zipcode'])[['alarm_source_description_tx']].count().reset_index()

In [88]:
median_response_travel_time_by_zip_month_year = fires_df.groupby(by = ['incident_year','incident_month','zipcode'])[['incident_response_seconds_qy','incident_travel_tm_seconds_qy']].median().reset_index()

In [89]:
fires_by_zipcode_month_year['incident_month'].min()

1

In [90]:
#from: https://data.cityofnewyork.us/Business/Zip-Code-Boundaries/i8iw-xf4u
shapefile = '../Data/ZIP_CODE_040114/ZIP_CODE_040114.shp'

In [91]:
geodf = gpd.read_file(shapefile)

In [92]:
geodf.head()

,ZIPCODE,BLDGZIP,PO_NAME,POPULATION,AREA,STATE,COUNTY,ST_FIPS,CTY_FIPS,URL,SHAPE_AREA,SHAPE_LEN,geometry
0,11436,0,Jamaica,18681.0,2.269930e+07,NY,Queens,36,081,http://www.usps.com/,0.0,0.0,"POLYGON ((1038098.251871482 188138.3800067157,..."
1,11213,0,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1001613.712964058 186926.4395172149,..."
2,11212,0,Brooklyn,83866.0,4.197210e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1011174.275535807 183696.33770971, 1..."
3,11225,0,Brooklyn,56527.0,2.369863e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((995908.3654508889 183617.6128015518,..."
4,11218,0,Brooklyn,72280.0,3.686880e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((991997.1134308875 176307.4958601296,..."


In [93]:
zipcode_df = geodf[['ZIPCODE','COUNTY','geometry','POPULATION']]

In [94]:
shp = fiona.open(shapefile)

In [95]:
zipcode_df['x'] = [[x[0] for x in feat["geometry"]["coordinates"][0]] for feat in shp]
zipcode_df['y'] = [[y[1] for y in feat["geometry"]["coordinates"][0]] for feat in shp]

C:\Users\noaha\Anaconda3\envs\dsi\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\noaha\Anaconda3\envs\dsi\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [96]:
zipcode_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 6 columns):
ZIPCODE       263 non-null object
COUNTY        263 non-null object
geometry      263 non-null object
POPULATION    263 non-null float64
x             263 non-null object
y             263 non-null object
dtypes: float64(1), object(5)
memory usage: 12.4+ KB


In [97]:
fires_by_zipcode_month_year_shp = fires_by_zipcode_month_year.merge(zipcode_df,left_on='zipcode',right_on='ZIPCODE')

In [102]:
fires_by_zipcode_month_year_shp.sort_values(by = 'alarm_source_description_tx',inplace=True)

## Create Bokeh Map

In [ ]:
# # creating a Bokeh map
# # Break tweets dataframe into two dataframes:
all_fires = fires_by_zipcode_month_year_shp
visible_fires = fires_by_zipcode_month_year_shp[(fires_by_zipcode_month_year_shp['incident_year']==fires_by_zipcode_month_year_shp['incident_year'].min()) & (fires_by_zipcode_month_year_shp['incident_month']==fires_by_zipcode_month_year_shp['incident_month'].min())]

# # make column data source dictionaries for each dataset for the bokeh map

visible_source = ColumnDataSource(data = dict(zipcode = visible_fires['zipcode'],
                                              year = visible_fires['incident_year'],
                                              month = visible_fires['incident_month'],
                                              fire_count = visible_fires['alarm_source_description_tx'],
                                              x = visible_fires['x'],
                                              y = visible_fires['y']
                                           )
)
all_source = ColumnDataSource(data = dict(zipcode = all_fires['zipcode'],
                                          year = all_fires['incident_year'],
                                          month = all_fires['incident_month'],
                                          fire_count = all_fires['alarm_source_description_tx'],
                                          x = all_fires['x'],
                                          y = all_fires['y']
                                         )
)

                             

# Sidebar tools for the bokeh map
TOOLS="hover,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select"

# setting the text to display when hovering over points on map
TOOLTIPS =[
    ('zipcode', '@zipcode'),
    ('fire count', '@fire_count'),
    ('Year','@year'),
    ('Month','@month')
]

# callback function
# Since Bokeh renders maps using Javascript, a callback function is needed to pull in our data
# to the map app, as python code cannot be referenced in javascript
callback = CustomJS(args=dict(all_source=all_source,visible_source = visible_source), code="""
    
    var all_data = all_source.data;
    var visible_data = visible_source.data;
    var month = month.value;
    var year = year.value;
    
    visible_data.fire_count = []
    visible_data.zipcode = []
    visible_data.month = []
    visible_data.year = []
    visible_data.x = []
    visible_data.y = []
    
    for(var i =0; i < all_data.x.length; i++) {
        if ((all_data.year[i] == year)&(all_data.month[i]==month)) {

            visible_data.fire_count.push(all_data.fire_count[i]);
            visible_data.zipcode.push(all_data.zipcode[i]);
            visible_data.month.push(all_data.month[i]);
            visible_data.year.push(all_data.year[i]);
            visible_data.x.push(all_data.x[i]);
            visible_data.y.push(all_data.y[i]);
        }
    }
    
    visible_source.change.emit();
""")
# Slider to control month of the year data is shown for
month_slider = Slider(start=1,end=12, value=1, step=1, title="Month", callback = callback)
callback.args['month'] = month_slider

# Slider to control year data is shown for
year_slider = Slider(start=fires_by_zipcode_month_year_shp['incident_year'].min(), end=fires_by_zipcode_month_year_shp['incident_year'].max(), value=2018, step=1,title="Year",callback = callback)
callback.args['year'] = year_slider

# Set Map Properties
p = figure(title='Map of Fire Incidents by Zipcode',plot_height = 500,tools = TOOLS, tooltips = TOOLTIPS)

# Add basemap
p.patches('x', 'y',
          source=visible_source,
          line_color = 'grey',
          fill_color = {'field': 'fire_count','transform':LogColorMapper(palette=palette)})


layout = column(year_slider,month_slider,p)
curdoc().add_root(layout)
output_file("../Maps/fire_map.html", title="fire_map.py")
output_notebook()
show(layout)  # open a browser

Loading BokehJS ...